In [108]:
import pandas as pd
import datetime 
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import keras

import keras_tuner
from sklearn.decomposition import PCA



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

SEED = 42

tf.keras.utils.set_random_seed(
    SEED
)

Num GPUs Available:  0


# Preprocessing

In [109]:
df_master = pd.read_csv("..\..\datasources\master_dataset.csv")

df_master

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.FOPR,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment
0,1955-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
1,1955-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
2,1955-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
3,1955-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
4,1955-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22845,2022-11-01,1.3625,20.0,6.0,6.0,12.0,2.0,-26.0,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,6.25,0.258793,0.719491,NaN,NaN
22846,2022-11-02,1.3632,33.0,6.0,10.0,13.0,14.0,-34.0,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,6.25,0.560365,0.948609,NaN,NaN
22847,2022-11-03,1.3712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,7.00,NaN,NaN,NaN,NaN
22848,2022-11-04,1.3746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.560856,17.131522,6.758784,24.702028,26.021942,NaN,NaN,NaN,NaN,NaN


In [110]:
df_master['y_exp'] = np.full(df_master.shape[0], -1)
for i, row in df_master.iterrows():
    try:
        two_weeks_from_current_date = datetime.datetime.strptime(row.date, "%Y-%m-%d") + datetime.timedelta(weeks=2)
        futur_rate = list(df_master.loc[df_master['date'] == str(two_weeks_from_current_date.date())].rate)[0]
        df_master.loc[i, 'y_exp' ] = futur_rate
    except: 
        break

df_master = df_master.loc[df_master.y_exp >= 0]
df_master

,date,rate,sent_fin_us_uncertainty,sent_fin_us_litigious,sent_fin_us_strong_modal,sent_fin_us_weak_modal,sent_fin_us_constraining,sent_fin_us_optimistic,sent_fin_can_uncertainty,sent_fin_can_litigious,...,index_WGTS.MTLS,index_WGTS.NATURALGAS,index_WGTS.WCC,index_WGTS.WTI,us_bank_rate,sent_bert_us_fin,sent_bert_us_gen,sent_bert_can_fin_sentiment,sent_bert_can_gen_sentiment,y_exp
7921,1981-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1895
7922,1981-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1863
7923,1981-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1876
7924,1981-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,.,NaN,NaN,NaN,NaN,1.1863
7927,1981-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.084779,11.247208,6.124090,12.893272,15.75,NaN,NaN,NaN,NaN,1.1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,2022-10-17,1.3877,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3615
22831,2022-10-18,1.3714,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3625
22832,2022-10-19,1.3741,13.0,9.0,2.0,7.0,5.0,-2.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3632
22833,2022-10-20,1.3767,20.0,6.0,6.0,12.0,2.0,-26.0,40.0,3.0,...,17.131522,6.758784,24.702028,26.021942,6.25,0.514009,0.97971,0.316318,0.703847,1.3712


In [111]:
df_master= df_master.dropna()
df_master= df_master.reset_index(drop=True)
df_master = df_master.loc[df_master[' us_bank_rate'] != '.']

## Dataset split

In [112]:
def split_train_test_val(df):
    # Split train val test datasets
    test_cutting_date = '2022-05-01'
    val_cutting_date = '2022-02-01'
    df_train = df[df['date'] < val_cutting_date]
    df_val = df[(df['date'] >= val_cutting_date) &
                (df['date'] < test_cutting_date)]
    df_test = df[df['date'] >= test_cutting_date]
    return df_train, df_val, df_test

train, val, test = split_train_test_val(df_master)
display(train.shape, val.shape, test.shape)

(6718, 46)

(63, 46)

(121, 46)

In [113]:
sorted_index = ['interest_rate_can',
 'index_WGTS.BRENT',
 'rate',
 'ti_rate_rsi_14',
 'sent_fin_can_litigious',
 'ti_rate_stochrsi_14',
 'sent_fin_us_strong_modal',
 'ti_rate_williams_%R',
 'index_WGTS.COAL',
 'ti_rate_macd_12_26',
 'sent_fin_us_uncertainty',
 'sent_fin_can_uncertainty',
 'index_W.FISH',
 'ti_rate_atr',
 'sent_fin_us_weak_modal',
 'sent_bert_can_fin_sentiment',
 'index_WGTS.WCC',
 'index_W.ENER',
 'sent_bert_us_gen',
 'sent_bert_can_gen_sentiment',
 'sent_fin_us_litigious',
 'sent_bert_us_fin',
 'index_W.FOPR',
 'ti_rate_utlimate_oscillator',
 'index_WGTS.FOPR',
 'sent_fin_can_strong_modal',
 'ti_rate_adx_14',
 'ti_rate_roc',
 'ti_rate_cci',
 'sent_fin_can_weak_modal',
 'sent_fin_can_optimistic',
 'sent_fin_us_optimistic',
 'index_WGTS.NATURALGAS',
 'sent_fin_can_constraining',
 'us_bank_rate',
 'index_WGTS.MTLS',
 'index_WGTS.AGRI',
 'index_W.MTLS',
 'index_W.BCNE',
 'index_WGTS.WTI',
 'index_WGTS.FISH',
 'sent_fin_us_constraining',
 'index_W.AGRI',
 'index_W.BCPI']

In [114]:
X_train = train[sorted_index[:5]].astype(float)
y_train = train.loc[:, train.columns == "y_exp"]

X_val = train[sorted_index[:5]].astype(float)
y_val = train.loc[:, train.columns == "y_exp"]

X_test = test[sorted_index[:5]].astype(float)
y_test = test.loc[:, train.columns == "y_exp"]

In [125]:

display(X_test.shape, y_test.shape)

(121, 5)

(121, 1)

# Baseline - Mean

In [116]:
baseline_y_pred = np.full(y_test.shape[0] , X_train.rate.mean())
mse = mean_squared_error(y_test.y_exp.to_list(), baseline_y_pred)
print(f" Mean squared error: {mse}")

 Mean squared error: 0.003181742138969571


# DNN Model

## Hyper-parameter tunning


In [19]:
def build_tune_model(hp):
	# create model
	input_dim = len(X_train.columns)
	model = Sequential()
	model.add(Dense( hp.Int("layer1", min_value=1, max_value=25, step=5), input_shape=(input_dim,), kernel_initializer='normal', activation='relu'))
    
	nb_layer = hp.Int("num_layers", 0, 4)
	if nb_layer != 0:
		for i in range(0, nb_layer):
			model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i}", min_value=1, max_value=25, step=5),
                    activation='relu',
                    kernel_initializer='normal'
                )
            )
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	# le = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
	le = hp.Choice('lr', [1.0, 1e-2, 1e-4, 1e-3])
	model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=0.001))
    
	return model

In [20]:
tuner = keras_tuner.RandomSearch(
    build_tune_model,
    objective='val_loss',
    overwrite=True,
    max_trials=10,
    project_name="DNN_Model_Tune")

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
                        verbose=1, mode='auto', restore_best_weights=True)

tuner.search(x=X_train, y=y_train, batch_size=5, epochs=100, validation_data=(X_val, y_val), callbacks=[monitor])
best_model = tuner.get_best_models()[0]
best_model.summary()

Trial 10 Complete [00h 00m 28s]
val_loss: 0.0003758674720302224

Best val_loss So Far: 0.0003737575316336006
Total elapsed time: 00h 03m 29s
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                66        
                                                                 
 dense_1 (Dense)             (None, 11)                132       
                                                                 
 dense_2 (Dense)             (None, 6)                 72        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
Total par

## Best model training

In [117]:
# define base model
def baseline_model(input_dim):
	# create model
	model = Sequential()
	model.add(Dense(5, input_shape=(input_dim,), kernel_initializer='normal', activation='relu'))
	model.add(Dense(5,  kernel_initializer='normal', activation='relu'))


	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=0.001))
	# model.compile(loss='mean_squared_error', optimizer='adam')
    
	return model

In [122]:
import shutil
nb_run = 5
for run in range(nb_run):
    model = baseline_model(X_train.shape[1])

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                            verbose=1, mode='auto', restore_best_weights=True)

    model.fit(x=X_train, y=y_train, batch_size=1, epochs=100, validation_data=(X_val, y_val), callbacks=[monitor])
    model.save(f"Model_DNN_{run}")
    # shutil.make_archive(f"Model_DNN_{run}", 'zip', f"Model_DNN_{run}")

    dnn_y_pred = [p[0] for p in model.predict(X_test)]

    mse = mean_squared_error(y_test.y_exp.to_list(), dnn_y_pred)
    print(f"Model_DNN_{run}: Mean squared error: {mse}")

Epoch 1/100
6718/6718 [==============================] - 9s 1ms/step - loss: 0.0490 - val_loss: 0.0105
Epoch 2/100
6718/6718 [==============================] - 8s 1ms/step - loss: 0.0058 - val_loss: 0.0030
Epoch 3/100
6718/6718 [==============================] - 8s 1ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 4/100
6718/6718 [==============================] - 8s 1ms/step - loss: 0.0014 - val_loss: 7.5074e-04
Epoch 5/100
6718/6718 [==============================] - 9s 1ms/step - loss: 8.7309e-04 - val_loss: 7.1859e-04
Epoch 6/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.7564e-04 - val_loss: 4.0220e-04
Epoch 7/100
6718/6718 [==============================] - 8s 1ms/step - loss: 8.1802e-04 - val_loss: 0.0024
Epoch 8/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.7436e-04 - val_loss: 4.8078e-04
Epoch 9/100
6718/6718 [==============================] - 8s 1ms/step - loss: 7.8863e-04 - val_loss: 4.4968e-04
Epoch 10/100
6718/6718 [=========

In [118]:
model = baseline_model(X_train.shape[1])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                        verbose=1, mode='auto', restore_best_weights=True)

model.fit(x=X_train, y=y_train, batch_size=1, epochs=100, validation_data=(X_val, y_val), callbacks=[monitor])


Epoch 1/100
6718/6718 [==============================] - 9s 1ms/step - loss: 0.0438 - val_loss: 0.0063
Epoch 2/100
6718/6718 [==============================] - 8s 1ms/step - loss: 0.0051 - val_loss: 0.0029
Epoch 3/100
6718/6718 [==============================] - 9s 1ms/step - loss: 0.0025 - val_loss: 0.0020
Epoch 4/100
6718/6718 [==============================] - 8s 1ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 5/100
6718/6718 [==============================] - 8s 1ms/step - loss: 8.1783e-04 - val_loss: 5.3152e-04
Epoch 6/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.4400e-04 - val_loss: 3.8475e-04
Epoch 7/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.8973e-04 - val_loss: 0.0014
Epoch 8/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.4954e-04 - val_loss: 5.1335e-04
Epoch 9/100
6718/6718 [==============================] - 9s 1ms/step - loss: 7.5909e-04 - val_loss: 4.4904e-04
Epoch 10/100
6718/6718 [=============

## Evaluation

In [127]:
model = keras.models.load_model("model_4")


In [128]:
dnn_y_pred = [p[0] for p in model.predict(X_test)]

mse = mean_squared_error(y_test.y_exp.to_list(), dnn_y_pred)
print(f" Mean squared error: {mse}")


4/4 [==============================] - 0s 1ms/step
 Mean squared error: 0.0005673522286965157


In [18]:
# import shutil
# model.save("Model_DNN_5-5-1")
# shutil.make_archive("Model_DNN_5-5-1", 'zip', "Model_DNN_5-5-1")

INFO:tensorflow:Assets written to: Model_DNN_5-5-1\assets


'e:\\Personal\\GitHub\\CHAMOAnalyticsDatathon2022\\modeling\\DNN\\Model_DNN_5-5-1.zip'

In [173]:
all_pred = []

for i in range(5):

    model = keras.models.load_model(f"model_{i}")

    dnn_y_pred = [p[0] for p in model.predict(X_test)]

    all_pred.append(dnn_y_pred[14:])

    mse = mean_squared_error(y_test.y_exp.to_list(), dnn_y_pred)
    print(f" Mean squared error: {mse}")

all_pred = np.array(all_pred)

4/4 [==============================] - 0s 1ms/step
 Mean squared error: 0.000650012557642633
4/4 [==============================] - 0s 2ms/step
 Mean squared error: 0.0012821807020339492
4/4 [==============================] - 0s 666us/step
 Mean squared error: 0.0007922886947694251
4/4 [==============================] - 0s 1ms/step
 Mean squared error: 0.0007277919064569916
4/4 [==============================] - 0s 1ms/step
 Mean squared error: 0.0005673522286965157


In [192]:
pmin = all_pred.min(axis=0)
pmax = all_pred.max(axis=0)

drawdown = (pmax-pmin) / pmax * 100
print(f"Mean Drawdown: {np.mean(drawdown)}")

bias = np.abs(np.mean(all_pred) - test.y_exp.to_numpy())
print(f"Mean Bias: {np.mean(bias)}")

variance = np.var(all_pred)
print(f"variance: {variance}")



Mean Drawdown: 3.665335178375244
Mean Bias: 0.029616780407172583
variance: 0.001693530473858118


# Vizualisation

In [134]:
import plotly.graph_objects as go


In [135]:
import plotly.graph_objects as go

two_week_dates = pd.to_datetime(test['date'], format="%Y-%m-%d") + datetime.timedelta(weeks=2)

fig = go.Figure()

fig.add_scatter(x=two_week_dates, y=test.y_exp, mode='lines', name="Taux d'échange USD/CAD réel")
fig.add_scatter(x=two_week_dates, y=baseline_y_pred, mode='lines', name="Baseline - Mean")
fig.add_scatter(x=two_week_dates, y=dnn_y_pred, mode='lines', name="DNN-5-5-1")

fig.update_layout(
    title="Comparaison du taux de change réel et du prédit",
    xaxis_title="Date",
    yaxis_title="Taux d'échange USD/CAD",
    legend_title="Legend",
    font=dict(
        size=18,
    ),
    width=1600, height=800,
)

fig.show()